## Model Building and Training for 2nd Proactive Strategy

In [1]:
# Importing necessary libraries for model building and training
import os
import gc
import utils
import numpy as np
import pandas as pd
import csv
import tensorflow as tf
from tensorflow.keras import models, layers, regularizers
from tensorflow.keras import callbacks
from tabulate import tabulate

#### Loading the sequences

In [2]:
# Defining the paths
base_path = "data/processed"
datasets = ["cj", "sj", "tj"]
splits = ["train", "valid"]

In [3]:
# Load the sequence arrays for model training
loaded_data = {data: [utils.load_sequences(base_path, data, split) for split in splits] for data in datasets}
(trd_cj, trl_cj), (vad_cj, val_cj) = loaded_data["cj"]
(trd_sj, trl_sj), (vad_sj, val_sj) = loaded_data["sj"]
(trd_tj, trl_tj), (vad_tj, val_tj) = loaded_data["tj"]

Loaded train data for cj: (4742, 30, 35), (4742, 6)
Loaded valid data for cj: (1707, 30, 35), (1707, 6)
Loaded train data for sj: (4742, 30, 35), (4742, 6)
Loaded valid data for sj: (1707, 30, 35), (1707, 6)
Loaded train data for tj: (4742, 30, 35), (4742, 6)
Loaded valid data for tj: (1707, 30, 35), (1707, 6)


#### Building the model

In [4]:
# Function to build the GRU model for multivariate time series forecasting
def build_multivariate_forecasting_model(input_shape, output_dim):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    model.add(layers.GRU(128, return_sequences=True, dropout=0.2))
    model.add(layers.GRU(64, dropout=0.2))
    model.add(layers.Dense(64, activation="relu"))
    model.add(layers.Dense(output_dim))
    model.compile(
        optimizer=tf.keras.optimizers.Nadam(learning_rate=0.001),
        loss="mse",
        metrics=["mae"]
    )
    model.summary()
    return model

In [5]:
# Setting the input shapes and output dims for the models
input_shape_cj = trd_cj.shape[1:]
input_shape_sj = trd_sj.shape[1:]
input_shape_tj = trd_tj.shape[1:]

output_dim_cj = trl_cj.shape[-1]
output_dim_sj = trl_sj.shape[-1]
output_dim_tj = trl_tj.shape[-1]

# Checking the input shapes of the datasets
print(f'Input shape for the model "Current joints" {input_shape_cj}')
print(f'Input shape for the model "Speed joints" {input_shape_sj}')
print(f'Input shape for the model "Temperature joints" {input_shape_tj}')

# Checking the output dims of the datasets
print(f'Output dims for the model "Current joints" {input_shape_cj}')
print(f'Output dims for the model "Speed joints" {input_shape_sj}')
print(f'Output dims for the model "Temperature joints" {input_shape_tj}')

Input shape for the model "Current joints" (30, 35)
Input shape for the model "Speed joints" (30, 35)
Input shape for the model "Temperature joints" (30, 35)
Output dims for the model "Current joints" (30, 35)
Output dims for the model "Speed joints" (30, 35)
Output dims for the model "Temperature joints" (30, 35)


In [6]:
# Building the multivariate time series forecasting model [regression]
model_cj = build_multivariate_forecasting_model(input_shape_cj, output_dim_cj)
utils.save_model_summary(model_cj, "models/cj/model_summary_cj.txt")

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru (GRU)                            │ (None, 30, 128)             │          63,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_1 (GRU)                          │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,158 (410.77 KB)

 Trainable params: 105,158 (410.77 KB)

 Non-trainable params: 0 (0.00 B)

Model summary saved at models/cj/model_summary_cj.txt


In [7]:
# Building the multivariate time series forecasting model [regression]
model_sj = build_multivariate_forecasting_model(input_shape_sj, output_dim_sj)
utils.save_model_summary(model_sj, "models/sj/model_summary_sj.txt")

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_2 (GRU)                          │ (None, 30, 128)             │          63,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_3 (GRU)                          │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,158 (410.77 KB)

 Trainable params: 105,158 (410.77 KB)

 Non-trainable params: 0 (0.00 B)

Model summary saved at models/sj/model_summary_sj.txt


In [8]:
# Building the multivariate time series forecasting model [regression]
model_tj = build_multivariate_forecasting_model(input_shape_tj, output_dim_tj)
utils.save_model_summary(model_tj, "models/tj/model_summary_tj.txt")

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ gru_4 (GRU)                          │ (None, 30, 128)             │          63,360 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ gru_5 (GRU)                          │ (None, 64)                  │          37,248 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 64)                  │           4,160 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 6)                   │             390 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 105,158 (410.77 KB)

 Trainable params: 105,158 (410.77 KB)

 Non-trainable params: 0 (0.00 B)

Model summary saved at models/tj/model_summary_tj.txt


#### Training the model

In [9]:
# Training the multivariate time series forecasting model [Regression]
history_cj = utils.train_model(model_cj, trd_cj, trl_cj, vad_cj, val_cj, epochs=15, batch_size=32, checkpoint_filename="models/cj/best_model_cj.keras")
utils.save_model(model_cj, "models/cj/model_cj.keras")
utils.save_training_history(history_cj, "models/cj/training_history_cj.csv")

Epoch 1/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 9s 29ms/step - loss: 0.2869 - mae: 0.3770 - val_loss: 0.1919 - val_mae: 0.2839 - learning_rate: 0.0010
Epoch 2/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 35ms/step - loss: 0.1852 - mae: 0.2956 - val_loss: 0.1906 - val_mae: 0.2874 - learning_rate: 0.0010
Epoch 3/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1827 - mae: 0.2923 - val_loss: 0.1914 - val_mae: 0.2839 - learning_rate: 0.0010
Epoch 4/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 25ms/step - loss: 0.1815 - mae: 0.2908 - val_loss: 0.1911 - val_mae: 0.2824 - learning_rate: 0.0010
Epoch 5/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 0.1799 - mae: 0.2891 - val_loss: 0.1883 - val_mae: 0.2810 - learning_rate: 0.0010
Epoch 6/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 0.1779 - mae: 0.2872 - val_loss: 0.1837 - val_mae: 0.2775 - learning_rate: 0.0010
Epoch 7/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.1736 - mae: 0.2831 - val_loss: 0.1678 - val_mae: 0.2705 - learning_rate: 0.0010

In [10]:
# Training the multivariate time series forecasting model [Regression]
history_sj = utils.train_model(model_sj, trd_sj, trl_sj, vad_sj, val_sj, epochs=15, batch_size=32, checkpoint_filename="models/sj/best_model_sj.keras")
utils.save_model(model_sj, "models/sj/model_sj.keras")
utils.save_training_history(history_sj, "models/sj/training_history_sj.csv")

Epoch 1/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 11s 32ms/step - loss: 0.0614 - mae: 0.1659 - val_loss: 0.0282 - val_mae: 0.0921 - learning_rate: 0.0010
Epoch 2/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0290 - mae: 0.0961 - val_loss: 0.0283 - val_mae: 0.0870 - learning_rate: 0.0010
Epoch 3/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 6s 37ms/step - loss: 0.0289 - mae: 0.0953 - val_loss: 0.0280 - val_mae: 0.0874 - learning_rate: 0.0010
Epoch 4/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0287 - mae: 0.0952 - val_loss: 0.0280 - val_mae: 0.0845 - learning_rate: 0.0010
Epoch 5/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 0.0281 - mae: 0.0935 - val_loss: 0.0268 - val_mae: 0.0836 - learning_rate: 0.0010
Epoch 6/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 33ms/step - loss: 0.0273 - mae: 0.0912 - val_loss: 0.0259 - val_mae: 0.0768 - learning_rate: 0.0010
Epoch 7/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 28ms/step - loss: 0.0260 - mae: 0.0875 - val_loss: 0.0252 - val_mae: 0.0786 - learning_rate: 0.001

In [11]:
# Training the multivariate time series forecasting model [Regression]
history_tj = utils.train_model(model_tj, trd_tj, trl_tj, vad_tj, val_tj, epochs=15, batch_size=32, checkpoint_filename="models/tj/best_model_tj.keras")
utils.save_model(model_tj, "models/tj/model_tj.keras")
utils.save_training_history(history_tj, "models/tj/training_history_tj.csv")

Epoch 1/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 10s 35ms/step - loss: 528.7366 - mae: 20.5664 - val_loss: 0.0757 - val_mae: 0.2279 - learning_rate: 0.0010
Epoch 2/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 24.2220 - mae: 3.4787 - val_loss: 0.0734 - val_mae: 0.2198 - learning_rate: 0.0010
Epoch 3/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 21.7807 - mae: 3.2177 - val_loss: 0.0689 - val_mae: 0.2083 - learning_rate: 0.0010
Epoch 4/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 36ms/step - loss: 20.6005 - mae: 3.0790 - val_loss: 0.0587 - val_mae: 0.1883 - learning_rate: 0.0010
Epoch 5/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - loss: 19.8836 - mae: 2.9879 - val_loss: 0.0499 - val_mae: 0.1701 - learning_rate: 0.0010
Epoch 6/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 4s 27ms/step - loss: 19.3771 - mae: 2.9219 - val_loss: 0.0434 - val_mae: 0.1563 - learning_rate: 0.0010
Epoch 7/15
149/149 ━━━━━━━━━━━━━━━━━━━━ 5s 34ms/step - loss: 18.9902 - mae: 2.8685 - val_loss: 0.0392 - val_mae: 0.1471 - learning_ra

In [12]:
# Freeing up memory
gc.collect()

2627